In [1]:
import os

In [2]:
%pwd

'/Users/nikhildeka/Desktop/MLflow/textsummariser/research'

In [3]:
os.chdir('../')
%pwd

'/Users/nikhildeka/Desktop/MLflow/textsummariser'

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from pathlib import Path
from src.TextSummarizer.logging import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists ")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2024-11-08 13:46:09,635: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-08 13:46:09,638: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-08 13:46:09,640: INFO: common: created directory at: artifacts]
[2024-11-08 13:46:09,641: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-08 13:46:12,940: INFO: 2540966342: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EA10:DAA91:22CC15:2AFAFE:672DC8CA
Accept-Ranges: bytes
Date: Fri, 08 Nov 2024 08:16:11 GMT
Via: 1.1 varnish
X-Served-By: cache-del21727-DEL
X-Cache: MI